### This example demonstrates how to use prompts as described at

In [3]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.8.0"

Installed Packages Microsoft.SemanticKernel, 1.8.0

### Step 3: Instantiate the Kernel

In [4]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

### 0.0 Initial prompt

In [6]:
string request = "I want to send an email to the marketing team celebrating their recent milestone.";
string prompt = $"What is the intent of this request? {request}";

/* Uncomment this code to make this example interactive
// <InitialPrompt>
Write("Your request: ");
string request = ReadLine()!;
string prompt = $"What is the intent of this request? {request}";
// </InitialPrompt>
*/

Console.WriteLine("0.0 Initial prompt");
// <InvokeInitialPrompt>
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

0.0 Initial prompt
The intent of this request is to draft an email to the marketing team in order to congratulate them on their recent milestone and celebrate their hard work and achievements.


### 1.0 Make the prompt more specific

In [8]:
// <MoreSpecificPrompt>
prompt = @$"What is the intent of this request? {request}
You can choose between SendEmail, SendMessage, CompleteTask, CreateDocument.";
// </MoreSpecificPrompt>

Console.WriteLine("1.0 Make the prompt more specific");
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

1.0 Make the prompt more specific
The intent of this request is to send an email to the marketing team celebrating their recent milestone. The appropriate action to choose would be SendEmail.


### // 2.0 Add structure to the output with formatting

In [9]:
// <StructuredPrompt>
prompt = @$"Instructions: What is the intent of this request?
Choices: SendEmail, SendMessage, CompleteTask, CreateDocument.
User Input: {request}
Intent: ";
// </StructuredPrompt>
 
Console.WriteLine("2.0 Add structure to the output with formatting");
Console.WriteLine(await kernel.InvokePromptAsync(prompt));

2.0 Add structure to the output with formatting
SendEmail


### 2.1 Add structure to the output with formatting (using Markdown and JSON)

In [10]:
  // <FormattedPrompt>
        prompt = $$"""
                 ## Instructions
                 Provide the intent of the request using the following format:
                 
                 ```json
                 {
                     "intent": {intent}
                 }
                 ```
                 
                 ## Choices
                 You can choose between the following intents:
                 
                 ```json
                 ["SendEmail", "SendMessage", "CompleteTask", "CreateDocument"]
                 ```
                 
                 ## User Input
                 The user input is:
                 
                 ```json
                 {
                     "request": "{{request}}"
                 }
                 ```
                 
                 ## Intent
                 """;
        // </FormattedPrompt>

    Console.WriteLine("2.1 Add structure to the output with formatting (using Markdown and JSON)");
    Console.WriteLine(await kernel.InvokePromptAsync(prompt));

2.1 Add structure to the output with formatting (using Markdown and JSON)
{
    "intent": "SendEmail"
}


### // 3.0 Provide examples with few-shot prompting

In [ ]:
// <FewShotPrompt>
        prompt = @$"Instructions: What is the intent of this request?
Choices: SendEmail, SendMessage, CompleteTask, CreateDocument.

User Input: Can you send a very quick approval to the marketing team?
Intent: SendMessage

User Input: Can you send the full update to the marketing team?
Intent: SendEmail

User Input: {request}
Intent: ";
        // </FewShotPrompt>

    Console.WriteLine("3.0 Provide examples with few-shot prompting");
    Console.WriteLine(await kernel.InvokePromptAsync(prompt));

### 4.0 Tell the AI what to do to avoid doing something wrong

In [ ]:
 // <AvoidPrompt>
        prompt = $"""
                 Instructions: What is the intent of this request?
                 If you don't know the intent, don't guess; instead respond with "Unknown".
                 Choices: SendEmail, SendMessage, CompleteTask, CreateDocument, Unknown.

                 User Input: Can you send a very quick approval to the marketing team?
                 Intent: SendMessage

                 User Input: Can you send the full update to the marketing team?
                 Intent: SendEmail

                 User Input: {request}
                 Intent: 
                 """;
        // </AvoidPrompt>

    Console.WriteLine("4.0 Tell the AI what to do to avoid doing something wrong");
    Console.WriteLine(await kernel.InvokePromptAsync(prompt));

### 5.0 Provide context to the AI

In [11]:
// <ContextPrompt>
        string history = """
                         User input: I hate sending emails, no one ever reads them.
                         AI response: I'm sorry to hear that. Messages may be a better way to communicate.
                         """;

        prompt = $"""
                 Instructions: What is the intent of this request?
                 If you don't know the intent, don't guess; instead respond with "Unknown".
                 Choices: SendEmail, SendMessage, CompleteTask, CreateDocument, Unknown.
                 
                 User Input: Can you send a very quick approval to the marketing team?
                 Intent: SendMessage
                 
                 User Input: Can you send the full update to the marketing team?
                 Intent: SendEmail
                 
                 {history}
                 User Input: {request}
                 Intent: 
                 """;
        // </ContextPrompt>

    Console.WriteLine("5.0 Provide context to the AI");
    Console.WriteLine(await kernel.InvokePromptAsync(prompt));

5.0 Provide context to the AI
SendEmail


### 6.0 Using message roles in chat completion prompts

In [ ]:
// <RolePrompt>
        history = """
                  <message role="user">I hate sending emails, no one ever reads them.</message>
                  <message role="assistant">I'm sorry to hear that. Messages may be a better way to communicate.</message>
                  """;

        prompt = $"""
                 <message role="system">Instructions: What is the intent of this request?
                 If you don't know the intent, don't guess; instead respond with "Unknown".
                 Choices: SendEmail, SendMessage, CompleteTask, CreateDocument, Unknown.</message>
                 
                 <message role="user">Can you send a very quick approval to the marketing team?</message>
                 <message role="system">Intent:</message>
                 <message role="assistant">SendMessage</message>
                 
                 <message role="user">Can you send the full update to the marketing team?</message>
                 <message role="system">Intent:</message>
                 <message role="assistant">SendEmail</message>
                 
                 {history}
                 <message role="user">{request}</message>
                 <message role="system">Intent:</message>
                 """;
        // </RolePrompt>

    Console.WriteLine("6.0 Using message roles in chat completion prompts");
        Console.WriteLine(await kernel.InvokePromptAsync(prompt));

###  7.0 Give your AI words of encouragement

In [ ]:
 // <BonusPrompt>
        history = """
                  <message role="user">I hate sending emails, no one ever reads them.</message>
                  <message role="assistant">I'm sorry to hear that. Messages may be a better way to communicate.</message>
                  """;

        prompt = $"""
                 <message role="system">Instructions: What is the intent of this request?
                 If you don't know the intent, don't guess; instead respond with "Unknown".
                 Choices: SendEmail, SendMessage, CompleteTask, CreateDocument, Unknown.
                 Bonus: You'll get $20 if you get this right.</message>
                
                 <message role="user">Can you send a very quick approval to the marketing team?</message>
                 <message role="system">Intent:</message>
                 <message role="assistant">SendMessage</message>
                
                 <message role="user">Can you send the full update to the marketing team?</message>
                 <message role="system">Intent:</message>
                 <message role="assistant">SendEmail</message>
                
                 {history}
                 <message role="user">{request}</message>
                 <message role="system">Intent:</message>
                 """;
        // </BonusPrompt>

        Console.WriteLine("7.0 Give your AI words of encouragement");
        Console.WriteLine(await kernel.InvokePromptAsync(prompt));

### This example demonstrates how to templatize prompts as described at
/// https://learn.microsoft.com/semantic-kernel/prompts/templatizing-prompts

In [16]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.8.0"
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;

  // Create a Semantic Kernel template for chat
        var chat = kernel.CreateFunctionFromPrompt(
            @"{{$history}}
            User: {{$request}}
            Assistant: ");
  // Create choices
        List<string> choices = ["ContinueConversation", "EndConversation"];

        // Create few-shot examples
        List<ChatHistory> fewShotExamples =
        [
            [
                new ChatMessageContent(AuthorRole.User, "Can you send a very quick approval to the marketing team?"),
                new ChatMessageContent(AuthorRole.System, "Intent:"),
                new ChatMessageContent(AuthorRole.Assistant, "ContinueConversation")
            ],
            [
                new ChatMessageContent(AuthorRole.User, "Thanks, I'm done for now"),
                new ChatMessageContent(AuthorRole.System, "Intent:"),
                new ChatMessageContent(AuthorRole.Assistant, "EndConversation")
            ]
        ];

        // Create handlebars template for intent
        var getIntent = kernel.CreateFunctionFromPrompt(
            new()
            {
                Template = """
                           <message role="system">Instructions: What is the intent of this request?
                           Do not explain the reasoning, just reply back with the intent. If you are unsure, reply with {{choices[0]}}.
                           Choices: {{choices}}.</message>

                           {{#each fewShotExamples}}
                               {{#each this}}
                                   <message role="{{role}}">{{content}}</message>
                               {{/each}}
                           {{/each}}

                           {{#each chatHistory}}
                               <message role="{{role}}">{{content}}</message>
                           {{/each}}

                           <message role="user">{{request}}</message>
                           <message role="system">Intent:</message>
                           """,
                TemplateFormat = "handlebars"
            },
            new HandlebarsPromptTemplateFactory()
        );

        ChatHistory history = [];

        // Start the chat loop
        while (true)
        {
            // Get user input
            Console.Write("User > ");
            var request = Console.ReadLine();

            // Invoke prompt
            var intent = await kernel.InvokeAsync(
                getIntent,
                new()
                {
                    { "request", request },
                    { "choices", choices },
                    { "history", history },
                    { "fewShotExamples", fewShotExamples }
                }
            );

            // End the chat if the intent is "Stop"
            if (intent.ToString() == "EndConversation")
            {
                break;
            }

            // Get chat response
            var chatResult = kernel.InvokeStreamingAsync<StreamingChatMessageContent>(
                chat,
                new()
                {
                    { "request", request },
                    { "history", string.Join("\n", history.Select(x => x.Role + ": " + x.Content)) }
                }
            );

            // Stream the response
            string message = "";
            await foreach (var chunk in chatResult)
            {
                if (chunk.Role.HasValue)
                {
                    Console.Write(chunk.Role + " > ");
                }

                message += chunk;
                Console.Write(chunk);
            }
            Console.WriteLine();

            // Append to history
            history.AddUserMessage(request!);
            history.AddAssistantMessage(message);
        }
    


Installed Packages Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.8.0

User > 

Error: Command cancelled.